# Implementation of the paper "Fine-grained generalized zero-shot learning via dense attribute-based attention"

In [1]:
import os,sys
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models.resnet as models
from PIL import Image
import h5py
import numpy as np
import scipy.io as sio
import pickle
import pdb
import matplotlib.pyplot as plt
import pandas as pd
import gensim.downloader as api
import torch.optim as optim
import importlib


# CUB dataset
#images = 17188

#classes = 200, 150 seen classes and 50 unseen classes

Each class has 312 number of attributes that represent the class infromation

In [3]:
img_dir = 'C:/Sushree/Jio_Institute/Dataset/'
print(img_dir)

file_paths = 'C:/Sushree/Jio_Institute/Dataset/data/xlsa17/data/CUB/res101.mat'
print(file_paths)

#resNet101.mat includes the following fields:
#-features: columns correspond to image instances
#-labels: label number of a class is its row number in allclasses.txt
#-image_files: image sources  

C:/Sushree/Jio_Institute/Dataset/
C:/Sushree/Jio_Institute/Dataset/data/xlsa17/data/CUB/res101.mat


# Let's visualize the data

In [ ]:
def visualize_data_distribution(file_paths):    
    matcontent = sio.loadmat(file_paths)
    print(matcontent)

    image_files = np.squeeze(matcontent['image_files'])
    #print(image_files)

    labels = np.squeeze(matcontent['labels'])
    print(labels)
    print(labels.size)  # 11788 for CUB

    class_names = []
    for idx in range(len(image_files)):
        image_file = image_files[idx][0]
        class_name = image_file.split('/')[5:][3]
        class_names.append(class_name)

    print(len(class_names))   
    #print(class_names)
    
    num_bins = 200 # # for CUB
    
    plt.figure(figsize=(35,6))
    
    plt.title("Data Distribution: CUB")
    plt.xlabel("Categories")
    plt.ylabel("Number of Classes")
    
    plt.xticks(rotation = 90)
    plt.grid(color = 'red', linestyle = '--', linewidth = 0.3)
    plt.hist(class_names, num_bins, align="mid")

visualize_data_distribution(file_paths)    

# Let's extract deep features (consider pre-trained ResNet 101 with no fine-tuning)

In [4]:
class CustomedDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, img_dir , file_paths, transform=None):
        self.matcontent = sio.loadmat(file_paths)
        self.image_files = np.squeeze(self.matcontent['image_files'])
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_file = self.image_files[idx][0]
        image_file = os.path.join(self.img_dir, '/'.join(image_file.split('/')[5:]))
        image = Image.open(image_file)
        
        if image.mode == 'L':
            image=image.convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

In [5]:
input_size = 224
data_transforms = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    
CUBDataset = CustomedDataset(img_dir , file_paths, data_transforms)


In [6]:
model_name = "resnet"

# Batch size for training (change depending on how much memory you have)
batch_size = 32

model_ref = models.resnet101(pretrained=True)
model_ref.eval()

model_f = nn.Sequential(*list(model_ref.children())[:-2])
model_f.eval()


for param in model_f.parameters():
    param.requires_grad = False
    
print(model_f)
        
from torchsummary import summary
summary(model_f, (3, 224, 224))    

dataset_loader = torch.utils.data.DataLoader(CUBDataset, batch_size=batch_size, shuffle=False, num_workers=0)

C:\Users\Sushree.Behera\anaconda3\envs\tf_sushree\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Sushree.Behera\anaconda3\envs\tf_sushree\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [7]:
all_features = []
for i_batch, imgs in enumerate(dataset_loader):
    print(i_batch)
    #pdb.set_trace()
    #imgs = imgs.to(device)
    features = model_f(imgs)
    all_features.append(features.numpy())
    
all_features = np.concatenate(all_features, axis=0)
print(all_features, all_features.shape)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# Let's extract semantic attributes of each category (consider pre-trained word2vec model with no fine-tuning)

In [9]:
print('Load pretrain w2v model')

model_name = 'word2vec-google-news-300'#best model
model = api.load(model_name)

dim_w2v = 300

#%%
replace_word = [('spatulate','broad'),('upperparts','upper parts'),('grey','gray')] # for CUB


path = 'C:/Sushree/Jio_Institute/Dataset/CUB_200_2011/CUB_200_2011/attributes/attributes.txt'
df=pd.read_csv(path,sep=' ',header = None, names = ['idx','des'])
des = df['des'].values

#%% filter
new_des = [' '.join(i.split('_')) for i in des]
new_des = [' '.join(i.split('-')) for i in new_des]
new_des = [' '.join(i.split('::')) for i in new_des]
new_des = [i.split('(')[0] for i in new_des]
new_des = [i[4:] for i in new_des]

#%% replace out of dictionary (OOD) words
for pair in replace_word:
    for idx,s in enumerate(des):
        des[idx] = s.replace(pair[0],pair[1])
print('Done replacing OOD words')

df['new_des'] = des
df.to_csv('C:/Sushree/Jio_Institute/Dataset/CUB_200_2011/CUB_200_2011/attributes/new_des.csv')
print('Done preprocessing attribute des')

Load pretrain w2v model
Done replacing OOD words
Done preprocessing attribute des


In [10]:

all_w2v = []
for s in new_des:
    print(s)
    words = s.split(' ')
    if words[-1] == '':     #remove empty element
        words = words[:-1]
    w2v = np.zeros(dim_w2v)
    for w in words:
        try:
            w2v += model[w]
        except Exception as e:
            print(e)
    all_w2v.append(w2v[np.newaxis,:])
    
#%%
all_w2v=np.concatenate(all_w2v,axis=0)
#pdb.set_trace()
#%%

with open('C:/Sushree/Jio_Institute/Dataset/CUB_200_2011/CUB_200_2011/w2v/CUB_attribute.pkl','wb') as f:
    pickle.dump(all_w2v,f)  

bill shape curved 
bill shape dagger
bill shape hooked
bill shape needle
bill shape hooked seabird
bill shape spatulate
"Key 'spatulate' not present"
bill shape all purpose
bill shape cone
bill shape specialized
wing color blue
wing color brown
wing color iridescent
wing color purple
wing color rufous
wing color grey
"Key 'grey' not present"
wing color yellow
wing color olive
wing color green
wing color pink
wing color orange
wing color black
wing color white
wing color red
wing color buff
upperparts color blue
"Key 'upperparts' not present"
upperparts color brown
"Key 'upperparts' not present"
upperparts color iridescent
"Key 'upperparts' not present"
upperparts color purple
"Key 'upperparts' not present"
upperparts color rufous
"Key 'upperparts' not present"
upperparts color grey
"Key 'upperparts' not present"
"Key 'grey' not present"
upperparts color yellow
"Key 'upperparts' not present"
upperparts color olive
"Key 'upperparts' not present"
upperparts color green
"Key 'upperparts' n

# Read the attributes and save as "w2v_att"

In [11]:
attribute_path = 'C:/Sushree/Jio_Institute/Dataset/CUB_200_2011/CUB_200_2011/w2v/CUB_attribute.pkl'

with open(attribute_path,'rb') as f:
    w2v_att = pickle.load(f)
assert w2v_att.shape == (312,300) # for AWA2
print('save w2v_att')

print(w2v_att, w2v_att.shape)

save w2v_att
[[-0.171875    0.4831543  -0.16796875 ... -0.29199219  0.16931152
  -0.11865234]
 [ 0.1484375   0.72070312  0.10888672 ... -0.30664062  0.23181152
   0.55566406]
 [ 0.05126953  0.4831543  -0.33251953 ... -0.31347656  0.41101074
  -0.21875   ]
 ...
 [ 0.17169189  0.44824219 -0.44018555 ... -0.40661621 -0.20285034
   0.43066406]
 [-0.02508545  0.26171875 -0.30932617 ... -0.24353027 -0.17749023
   0.38037109]
 [-0.02105713  0.12207031 -0.20239258 ... -0.1940918  -0.11444092
  -0.10351562]] (312, 300)


# Let's gather additional information (training, validation, and test indexes)

In [13]:
#%% get remaining metadata
matcontent = CUBDataset.matcontent
labels = matcontent['labels'].astype(int).squeeze() - 1

split_path = 'C:/Sushree/Jio_Institute/Dataset/data/xlsa17/data/CUB/att_splits.mat'
print(split_path)
    
#att_splits.mat includes the following fields:
#-att: columns correpond to class attribute vectors normalized to have unit l2 norm, following the classes order in allclasses.txt 
#-original_att: the original class attribute vectors without normalization
#-trainval_loc: instances indexes of train+val set features (for only seen classes) in resNet101.mat
#-test_seen_loc: instances indexes of test set features for seen classes
#-test_unseen_loc: instances indexes of test set features for unseen classes    


C:/Sushree/Jio_Institute/Dataset/data/xlsa17/data/CUB/att_splits.mat


In [14]:
def get_index_details(split_path):
    matcontent = sio.loadmat(split_path)
    print(matcontent)
    
    trainval_loc = matcontent['trainval_loc'].squeeze() - 1
    print(trainval_loc, len(trainval_loc))

    test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
    print(test_seen_loc, len(test_seen_loc))

    test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1
    print(test_unseen_loc, len(test_unseen_loc))
    
    att = matcontent['att'].T
    print(att, att.shape)
    
    original_att = matcontent['original_att'].T
    print(original_att, original_att.shape)
    return trainval_loc, test_seen_loc, test_unseen_loc, att, original_att
    
trainval_loc, test_seen_loc, test_unseen_loc, att, original_att = get_index_details(split_path)    

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Fri Aug 21 10:34:24 2020', '__version__': '1.0', '__globals__': [], 'allclasses_names': array([[array(['002.Laysan_Albatross'], dtype='<U20')],
       [array(['003.Sooty_Albatross'], dtype='<U19')],
       [array(['005.Crested_Auklet'], dtype='<U18')],
       [array(['007.Parakeet_Auklet'], dtype='<U19')],
       [array(['010.Red_winged_Blackbird'], dtype='<U24')],
       [array(['011.Rusty_Blackbird'], dtype='<U19')],
       [array(['012.Yellow_headed_Blackbird'], dtype='<U27')],
       [array(['013.Bobolink'], dtype='<U12')],
       [array(['015.Lazuli_Bunting'], dtype='<U18')],
       [array(['016.Painted_Bunting'], dtype='<U19')],
       [array(['017.Cardinal'], dtype='<U12')],
       [array(['018.Spotted_Catbird'], dtype='<U19')],
       [array(['019.Gray_Catbird'], dtype='<U16')],
       [array(['020.Yellow_breasted_Chat'], dtype='<U24')],
       [array(['021.Eastern_Towhee'], dtype='<U18')],
       [array(['022

# Save the feature map that includes ResNet50 features, labels, training and test (seen and unseen) data indexes, semantic attributes, and 

In [15]:

save_path = 'C:/Sushree/Jio_Institute/Dataset/CUB_200_2011/CUB_200_2011/feature_map_ResNet_101_CUB.hdf5'

f = h5py.File(save_path, "w")
f.create_dataset('feature_map', data=all_features,compression="gzip")
f.create_dataset('labels', data=labels,compression="gzip")
f.create_dataset('trainval_loc', data=trainval_loc,compression="gzip")
#    f.create_dataset('train_loc', data=train_loc,compression="gzip")
#    f.create_dataset('val_unseen_loc', data=val_unseen_loc,compression="gzip")
f.create_dataset('test_seen_loc', data=test_seen_loc,compression="gzip")
f.create_dataset('test_unseen_loc', data=test_unseen_loc,compression="gzip")
f.create_dataset('att', data=att,compression="gzip")
f.create_dataset('original_att', data=original_att,compression="gzip")
f.create_dataset('w2v_att', data=w2v_att,compression="gzip")
f.close()

In [ ]:
hf = h5py.File('C:/Sushree/Jio_Institute/Dataset/Animals_with_Attributes2/feature_map_ResNet_101_AWA2.hdf5', 'r')
features = np.array(hf.get('feature_map'))
print(features.shape)

In [ ]:
print(features)

In [ ]:
att = np.array(hf.get('att'))
print(att)


In [ ]:
f.close()

# Train the DAZLE model for AWA2

In [1]:
import os,sys
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models.resnet as models
from PIL import Image
import h5py
import numpy as np
import scipy.io as sio
import pickle
import pdb
import matplotlib.pyplot as plt
import pandas as pd
import gensim.downloader as api
import torch.optim as optim
import importlib


In [2]:
from DAZLE import DAZLE
from CUBDataLoader import CUBDataLoader
from helper_func import eval_zs_gzsl,visualize_attention#,get_attribute_attention_stats

In [3]:
data_path = 'C:/Sushree/Jio_Institute/Dataset/'
feature_path = 'C:/Sushree/Jio_Institute/Dataset/CUB_200_2011/CUB_200_2011/'
dataloader = CUBDataLoader(data_path, feature_path, device = None)


C:/Sushree/Jio_Institute/Dataset/
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
CUB
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Balance dataloader
_____
C:/Sushree/Jio_Institute/Dataset/CUB_200_2011/CUB_200_2011/feature_map_ResNet_101_CUB.hdf5
Expert Attr


In [4]:
dataloader.augment_img_path()

def get_lr(optimizer):
    lr = []
    for param_group in optimizer.param_groups:
        lr.append(param_group['lr'])
    return lr

In [5]:
seed = 214
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

batch_size = 32
nepoches = 100
niters = dataloader.ntrain * nepoches//batch_size
dim_f = 2048
dim_v = 300
init_w2v_att = dataloader.w2v_att # load the attribute features
att = dataloader.att
normalize_att = dataloader.normalize_att


trainable_w2v = True
lambda_ = 0.1
bias = 0
prob_prune = 0
uniform_att_1 = False
uniform_att_2 = False

seenclass = dataloader.seenclasses #load seen and unseen data
unseenclass = dataloader.unseenclasses
desired_mass = 1
report_interval = niters//nepoches

device = None

model = DAZLE(dim_f,dim_v,init_w2v_att,att,normalize_att,
            seenclass,unseenclass,
            lambda_,
            trainable_w2v,normalize_V=False,normalize_F=True,is_conservative=True,
            uniform_att_1=uniform_att_1,uniform_att_2=uniform_att_2,
            prob_prune=prob_prune,desired_mass=desired_mass, is_conv=False,
            is_bias=True)
model.to(device)

setup = {'pmp':{'init_lambda':0.1,'final_lambda':0.1,'phase':0.8},
         'desired_mass':{'init_lambda':-1,'final_lambda':-1,'phase':0.8}}
print(setup)

params_to_update = []
params_names = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        params_names.append(name)
        print("\t",name)
#%%
lr = 0.0001
weight_decay = 0.0001
momentum = 0.9
#%%
lr_seperator = 1
lr_factor = 1
print('default lr {} {}x lr {}'.format(params_names[:lr_seperator],lr_factor,params_names[lr_seperator:]))
optimizer  = optim.RMSprop( params_to_update ,lr=lr,weight_decay=weight_decay, momentum=momentum)
print('-'*30)
print('learing rate {}'.format(lr))
print('trainable V {}'.format(trainable_w2v))
print('lambda_ {}'.format(lambda_))
print('optimized seen only')
print('optimizer: RMSProp with momentum = {} and weight_decay = {}'.format(momentum,weight_decay))
print('-'*30)

------------------------------
Configuration
loss_type CE
no constraint V
normalize F
training to exclude unseen class [seen upperbound]
Init word2vec
Linear model
loss_att BCEWithLogitsLoss()
Bilinear attention module
******************************
Measure w2v deviation
Compute Pruning loss Parameter containing:
tensor(0)
Add one smoothing
Second layer attenion conditioned on image features
------------------------------
No sigmoid on attr score
{'pmp': {'init_lambda': 0.1, 'final_lambda': 0.1, 'phase': 0.8}, 'desired_mass': {'init_lambda': -1, 'final_lambda': -1, 'phase': 0.8}}
	 V
	 W_1
	 W_2
	 W_3
default lr ['V'] 1x lr ['W_1', 'W_2', 'W_3']
------------------------------
learing rate 0.0001
trainable V True
lambda_ 0.1
optimized seen only
optimizer: RMSProp with momentum = 0.9 and weight_decay = 0.0001
------------------------------


C:\Users\Sushree.Behera\Sushree\GZSL_Implementation\DAZLE.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.init_w2v_att = F.normalize(torch.tensor(init_w2v_att))
C:\Users\Sushree.Behera\Sushree\GZSL_Implementation\DAZLE.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.att = nn.Parameter(F.normalize(torch.tensor(att)),requires_grad = False)


In [6]:
best_performance = [0,0,0,0]
for i in range(0,niters):
    model.train()
    optimizer.zero_grad()
    
    batch_label, batch_feature, batch_att = dataloader.next_batch(batch_size)
    out_package = model(batch_feature)
    
    in_package = out_package
    in_package['batch_label'] = batch_label
    
    out_package=model.compute_loss(in_package)
    loss,loss_CE,loss_cal = out_package['loss'],out_package['loss_CE'],out_package['loss_cal']
    
    loss.backward()
    optimizer.step()
    if i%report_interval==0:
        print('-'*30)
        acc_seen, acc_novel, H, acc_zs = eval_zs_gzsl(dataloader,model,device,bias_seen=-bias,bias_unseen=bias)
        
        if H > best_performance[2]:
            best_performance = [acc_seen, acc_novel, H, acc_zs]
        stats_package = {'iter':i, 'loss':loss.item(), 'loss_CE':loss_CE.item(),
                         'loss_cal': loss_cal.item(),
                         'acc_seen':best_performance[0], 'acc_novel':best_performance[1], 'H':best_performance[2], 'acc_zs':best_performance[3]}
        
        print(stats_package)

------------------------------
bias_seen 0 bias_unseen 0
{'iter': 0, 'loss': 5.437799453735352, 'loss_CE': 5.403890132904053, 'loss_cal': 0.3390955328941345, 'acc_seen': 0, 'acc_novel': 0, 'H': 0, 'acc_zs': 0}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 220, 'loss': 1.7292962074279785, 'loss_CE': 1.6393898725509644, 'loss_cal': 0.8990627527236938, 'acc_seen': 0.3866586685180664, 'acc_novel': 0.49620482325553894, 'H': 0.43463547436261807, 'acc_zs': 0.5499619245529175}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 440, 'loss': 1.318319320678711, 'loss_CE': 1.2152273654937744, 'loss_cal': 1.0309193134307861, 'acc_seen': 0.48445358872413635, 'acc_novel': 0.5386620759963989, 'H': 0.5101217483505728, 'acc_zs': 0.6094498634338379}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 660, 'loss': 1.0087454319000244, 'loss_CE': 0.8710147738456726, 'loss_cal': 1.377306342124939, 'acc_seen': 0.5309994220733643, 'acc_novel': 0.5282782912254333

------------------------------
bias_seen 0 bias_unseen 0
{'iter': 6600, 'loss': 0.5836837291717529, 'loss_CE': 0.4318195879459381, 'loss_cal': 1.5186411142349243, 'acc_seen': 0.5924230217933655, 'acc_novel': 0.5670598745346069, 'H': 0.5794640446589812, 'acc_zs': 0.6518011689186096}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 6820, 'loss': 0.4956952929496765, 'loss_CE': 0.3623758852481842, 'loss_cal': 1.3331938982009888, 'acc_seen': 0.5924230217933655, 'acc_novel': 0.5670598745346069, 'H': 0.5794640446589812, 'acc_zs': 0.6518011689186096}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 7040, 'loss': 0.4279676675796509, 'loss_CE': 0.2710740566253662, 'loss_cal': 1.5689358711242676, 'acc_seen': 0.5924230217933655, 'acc_novel': 0.5670598745346069, 'H': 0.5794640446589812, 'acc_zs': 0.6518011689186096}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 7260, 'loss': 0.5017527341842651, 'loss_CE': 0.36248403787612915, 'loss_cal': 1.39268

------------------------------
bias_seen 0 bias_unseen 0
{'iter': 12980, 'loss': 0.4912301003932953, 'loss_CE': 0.3429970145225525, 'loss_cal': 1.4823307991027832, 'acc_seen': 0.5924230217933655, 'acc_novel': 0.5670598745346069, 'H': 0.5794640446589812, 'acc_zs': 0.6518011689186096}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 13200, 'loss': 0.3446468114852905, 'loss_CE': 0.18699601292610168, 'loss_cal': 1.5765081644058228, 'acc_seen': 0.5924230217933655, 'acc_novel': 0.5670598745346069, 'H': 0.5794640446589812, 'acc_zs': 0.6518011689186096}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 13420, 'loss': 0.3499443531036377, 'loss_CE': 0.12199143320322037, 'loss_cal': 2.279529094696045, 'acc_seen': 0.5924230217933655, 'acc_novel': 0.5670598745346069, 'H': 0.5794640446589812, 'acc_zs': 0.6518011689186096}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 13640, 'loss': 0.31113436818122864, 'loss_CE': 0.1510077565908432, 'loss_cal': 1.

------------------------------
bias_seen 0 bias_unseen 0
{'iter': 19360, 'loss': 0.38632136583328247, 'loss_CE': 0.23413819074630737, 'loss_cal': 1.521831750869751, 'acc_seen': 0.5924230217933655, 'acc_novel': 0.5670598745346069, 'H': 0.5794640446589812, 'acc_zs': 0.6518011689186096}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 19580, 'loss': 0.29094448685646057, 'loss_CE': 0.14028918743133545, 'loss_cal': 1.5065529346466064, 'acc_seen': 0.5924230217933655, 'acc_novel': 0.5670598745346069, 'H': 0.5794640446589812, 'acc_zs': 0.6518011689186096}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 19800, 'loss': 0.32226723432540894, 'loss_CE': 0.10400226712226868, 'loss_cal': 2.182649850845337, 'acc_seen': 0.5924230217933655, 'acc_novel': 0.5670598745346069, 'H': 0.5794640446589812, 'acc_zs': 0.6518011689186096}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 20020, 'loss': 0.27921462059020996, 'loss_CE': 0.12300543487071991, 'loss_cal'